In [0]:
import os
import sys
sys.path.append(f"{os.getcwd()}/..")

from pyspark.sql.types import StructType
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from pyspark.sql.functions import concat, regexp_extract, date_format, current_timestamp
from pyspark.sql.functions import when, col, to_timestamp, to_date, regexp_replace
from pyspark.sql.functions import sha2, concat_ws, coalesce, lit, trim, struct
from pyspark.sql.functions import to_json, expr, to_utc_timestamp
from pyspark.sql.types import StructType, ArrayType
from pyspark.sql.functions import transform
import re
from functions.utility import create_table_if_not_exists, get_function
from functions.transform import silver_scd2_transform
from functions.read import read_table
from functions.write import batch_scd2_write

catalog = dbutils.widgets.get('catalog')
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

settings = {
    "read_function": "functions.read.read_table",
    "transform_function": "functions.transform.silver_scd2_transform",
    "write_function": "functions.write.batch_scd2_write",
    "src_table_name": "edsm.bronze.systemsPopulated",
    "dst_table_name": "edsm.silver.systemsPopulated",
    "build_history": "false",
    "business_key": [
        "id"
    ],
    "surrogate_key": [
        "allegiance",
        "controllingFaction",
        "economy",
        "factions",
        "government",
        "population",
        "security",
        "state",
        "stations"
    ],
    "use_row_hash": True,
    "row_hash_col": "row_hash",
    "data_type_map": {
        "date": "timestamp"
    },
    "readStreamOptions": {
        "startingVersion": 5
    },
    "writeStreamOptions": {
        "mergeSchema": "false",
        "checkpointLocation": f"/Volumes/{catalog}/silver/utility/systemsPopulated/_checkpoints/",
        "delta.columnMapping.mode": "name"
    },
    "ingest_time_column": "derived_ingest_time"
}

df = read_table(spark, settings)
df = silver_scd2_transform(df, settings, spark)

df.select("row_hash").limit(10).show()

batch_scd2_write(df, settings, spark)